## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.2078,-159.7750,84.25,74,15,8.05,few clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,42.31,98,90,6.29,moderate rain
2,2,Naryan-Mar,RU,67.6713,53.0870,5.54,97,81,6.13,broken clouds
3,3,Mataura,NZ,-46.1927,168.8643,56.48,54,57,13.40,broken clouds
4,4,Kloulklubed,PW,7.0419,134.2556,82.90,76,66,6.44,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.2078,-159.7750,84.25,74,15,8.05,few clouds
4,4,Kloulklubed,PW,7.0419,134.2556,82.90,76,66,6.44,broken clouds
7,7,Atuona,PF,-9.8000,-139.0333,78.94,78,20,16.02,light rain
13,13,Maroantsetra,MG,-15.4333,49.7333,75.58,90,74,1.54,light rain
15,15,Alofi,NU,-19.0595,-169.9187,81.19,74,51,5.73,broken clouds
18,18,Labuhan,ID,-6.8844,112.2051,79.61,82,85,4.63,overcast clouds
20,20,Kahului,US,20.8947,-156.4700,82.42,70,40,25.32,scattered clouds
21,21,Saint George,US,37.1041,-113.5841,77.63,16,0,9.22,clear sky
26,26,Rikitea,PF,-23.1203,-134.9692,77.52,71,1,14.29,light rain
38,38,Hilo,US,19.7297,-155.0900,84.27,91,100,6.91,moderate rain


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.count()

City_ID                220
City                   220
Country                220
Lat                    220
Lng                    220
Max Temp               220
Humidity               220
Cloudiness             220
Wind Speed             220
Current Description    220
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,84.25,few clouds,-21.2078,-159.7750,
4,Kloulklubed,PW,82.90,broken clouds,7.0419,134.2556,
7,Atuona,PF,78.94,light rain,-9.8000,-139.0333,
13,Maroantsetra,MG,75.58,light rain,-15.4333,49.7333,
15,Alofi,NU,81.19,broken clouds,-19.0595,-169.9187,
18,Labuhan,ID,79.61,overcast clouds,-6.8844,112.2051,
20,Kahului,US,82.42,scattered clouds,20.8947,-156.4700,
21,Saint George,US,77.63,clear sky,37.1041,-113.5841,
26,Rikitea,PF,77.52,light rain,-23.1203,-134.9692,
38,Hilo,US,84.27,moderate rain,19.7297,-155.0900,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.

    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
new_hotel_df = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = new_hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,84.25,few clouds,-21.2078,-159.7750,Paradise Inn
4,Kloulklubed,PW,82.90,broken clouds,7.0419,134.2556,Storyboard Beach Resort
7,Atuona,PF,78.94,light rain,-9.8000,-139.0333,Villa Enata
13,Maroantsetra,MG,75.58,light rain,-15.4333,49.7333,Annexe Vatsy Hôtel
15,Alofi,NU,81.19,broken clouds,-19.0595,-169.9187,Taloa Heights
...,...,...,...,...,...,...,...
704,Yarada,IN,82.72,broken clouds,17.6500,83.2667,Yarada Jungle Beach Resorts
710,Masiga,PH,80.91,scattered clouds,13.3607,121.8278,Golden SunSet Resort
713,Banjar,ID,81.70,broken clouds,-8.1900,114.9675,The Hamsa Resort
716,Mahibadhoo,MV,83.61,broken clouds,3.7833,72.9667,Omadhoo Inn


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))